# 🤖 Hand Gesture Recognition System
---

## 1. 📦 Import Necessary Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import nbimporter
from data_preprocessing import DATA_DIR

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import TimeDistributed, Dense, Input, GlobalAveragePooling2D, BatchNormalization, Flatten, Bidirectional, GRU, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

---
## 2. 🗂️ Load Frame Sequences and Create TensorFlow Dataset

In [ ]:
# Load a sequence of image frames from a given folder
def load_sequence_frames(sample_path, seq_len=15, step=2, size=(224, 224)):
    frames = []

    # Get all PNG files sorted alphabetically
    image_files = sorted(sample_path.glob('*.png'))

    # Select frames with a stride (e.g. every 2nd frame)
    selected_files = image_files[::step][:seq_len]

    for img_path in selected_files:
        img = cv2.imread(str(img_path))                  # Read image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)       # Convert BGR to RGB
        img = cv2.resize(img, size)                      # Resize image to desired size
        frames.append(img)                               # Append to list

    # Normalize pixel values to [0, 1]
    frames = np.array(frames, dtype=np.float32) / 255.0
    return frames  # Shape: (seq_len, 224, 224, 3)


# Create a TensorFlow dataset from a root directory
def data_generator(root_dir):
    root = Path(root_dir)

    # Get class names (subdirectories)
    classes = sorted([d.name for d in root.iterdir() if d.is_dir()])
    print(classes)

    # Map class names to numeric labels
    class_to_idx = {c: i for i, c in enumerate(classes)}

    samples, labels = [], []

    # Collect all sample folders and their labels
    for c in classes:
        for sample_folder in (root / c).iterdir():
            if sample_folder.is_dir():
                samples.append(sample_folder)
                labels.append(class_to_idx[c])

    # Generator function that yields (sequence, label) pairs
    def gen():
        for sample_path, label in zip(samples, labels):
            seq = load_sequence_frames(sample_path)
            yield seq, label

    # Wrap the generator in a tf.data.Dataset
    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(15, 224, 224, 3), dtype=tf.float32),  # Input shape
            tf.TensorSpec(shape=(), dtype=tf.int32)                    # Label
        )
    )


---
## 3. 📊 Create Training and Validation Datasets

In [ ]:
train_dataset = (data_generator(f"{DATA_DIR}/train")
    .shuffle(100)
    .batch(4)
    .repeat()
    .prefetch(tf.data.AUTOTUNE))

In [ ]:
val_dataset = (data_generator(f"{DATA_DIR}/val")
    .batch(4)
    .prefetch(tf.data.AUTOTUNE))

---
## 4. 🧠 Build the MobileNetV2 + Bidirectional GRU Model

In [ ]:
num_classes = 5 
sequence_length = 15 # Each input sample is a sequence of 15 frames (images)
image_size = (224, 224, 3) # Size of each image frame (224x224 RGB)

# Define model input: a sequence of 15 images
inputs = Input(shape=(sequence_length, *image_size))

# Load MobileNetV2 as the base CNN to extract features from each frame
# - include_top=False: we remove the final classification layer since it doesn't match ours 
# - weights='imagenet': use pretrained weights
cnn_base = MobileNetV2(include_top=False, weights='imagenet', input_shape=image_size)

# Make CNN layers trainable (fine-tuning)
cnn_base.trainable = True

# Apply the CNN to each frame independently using TimeDistributed
x = TimeDistributed(cnn_base)(inputs)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(GlobalAveragePooling2D())(x)  # Convert CNN output to 1D vector per frame

# Pass the sequence of vectors to a Bidirectional GRU to learn temporal patterns
x = Bidirectional(GRU(128), kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.25)(x)  # Regularization to prevent overfitting

# Fully connected layer 
x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.25)(x)

# Final output layer with softmax for multi-class classification
outputs = Dense(num_classes, activation='softmax')(x)

# Build the model
model = Model(inputs, outputs)

# Compile the model with Adam optimizer and cross-entropy loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',  # since labels are integers and not hot encode , we use sparse_categorical_crossentropy
    metrics=['accuracy']
)

# Print the model summary
model.summary()


---
## 5. 🔢 Compute Dataset Sizes

In [ ]:
original_train_dataset = data_generator(f"{DATA_DIR}/train")
num_train_samples = sum(1 for _ in original_train_dataset)

original_val_dataset = data_generator(f"{DATA_DIR}/val")
num_val_samples = sum(1 for _ in original_val_dataset)

# Compute the number of steps per training epoch (batch size = 4)
steps_per_epoch = num_train_samples // 4

# Compute the number of validation steps per epoch (batch size = 4)
validation_steps = num_val_samples // 4

---
## 6. 🏋️ Train the Model with Callbacks

In [ ]:
# Stop training early if validation loss doesn't improve for 10 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Save the model only when it achieves the best validation accuracy
checkpoint = ModelCheckpoint(
    f"{DATA_DIR}/Outputs/best_model.keras", 
    save_best_only=True, 
    monitor='val_accuracy'
)

# Log training and validation metrics to a CSV file
csv_logger = CSVLogger(f"{DATA_DIR}/Outputs/training_log.csv", append=False)

# Reduce learning rate if validation loss stops improving for 3 epochs
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',   # Watch the validation loss
    factor=0.5,           # Reduce LR by half
    patience=3,           # Wait 3 epochs before reducing
    verbose=1,            # Print info when LR is reduced
    min_lr=1e-7           # Set a minimum bound for LR
)

# Train the model with the above callbacks
history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    epochs=40,
    callbacks=[early_stop, checkpoint, csv_logger, reduce_lr]
)


---
## 7. 📈 Plot Training and Validation Metrics

In [ ]:
history = pd.read_csv(f"{DATA_DIR}/Outputs/training_log.csv")

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Train Loss', marker='o')
plt.plot(history['val_loss'], label='Val Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()